# <center>Pydantic Output Parser

### - What is `PydanticOutputParser` in LangChain?

`PydanticOutputParser` is a structured output parser in LangChain that uses <b>Pydantic models</b> to enforce schema validation when processing LLM responses.

#### Why use `PydanticOutputParser` ?

- <b>Strict Schema Enforcement -> </b> Ensures that LLM responses follow a welll-defined structure.
- <b>Type Safety -> </b> Automatically converts LLM outputs into Python Objects.
- <b>Easy Validation -> </b> Uses Pydantic's build-in validatoin to catch incorrect or missing data
- <b>Seamless Integration -> </b> Works well with other LangChain components.

In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from dotenv import load_dotenv
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field
from typing import List

In [2]:
load_dotenv()

True

In [3]:
API_KEY = os.environ['GEMINI_API_KEY']
MODEL_NAME = 'gemini-2.0-flash'

In [4]:
model = ChatGoogleGenerativeAI(api_key=API_KEY, model=MODEL_NAME)

In [19]:
class Person(BaseModel):
    name: str = Field(description="Name of the Person")
    age: int = Field(description="Age of the Person")
    city: str = Field(description="Name of the city person belongs to")

In [20]:
class PersonList(BaseModel):
    people: List[Person]  # This will allow you to parse a list of Person objects

In [21]:
parser = PydanticOutputParser(pydantic_object=PersonList)

In [22]:
parser

PydanticOutputParser(pydantic_object=<class '__main__.PersonList'>)

In [23]:
template = PromptTemplate(
    template = 'Write me name, age, city of all of the gang member of van der linde gang \n {format_instruction}',
    input_variables=[],
    partial_variables={"format_instruction": parser.get_format_instructions()}
)

In [24]:
template

PromptTemplate(input_variables=[], input_types={}, partial_variables={'format_instruction': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"$defs": {"Person": {"properties": {"name": {"description": "Name of the Person", "title": "Name", "type": "string"}, "age": {"description": "Age of the Person", "title": "Age", "type": "integer"}, "city": {"description": "Name of the city person belongs to", "title": "City", "type": "string"}}, "required": ["name", "age", "city"], "title": "Person", "type": "object"}}, "properties": {"people": {"items": {"$ref": "#/$defs/Person"}, "title": "Pe

In [25]:
chain = template | model | parser

In [26]:
result = chain.invoke({})

In [27]:
result

PersonList(people=[Person(name='Arthur Morgan', age=36, city='Saint Louis, Missouri'), Person(name='Dutch van der Linde', age=55, city='Unspecified'), Person(name='Hosea Matthews', age=70, city='New York City, New York'), Person(name='John Marston', age=26, city='Unspecified'), Person(name='Abigail Roberts', age=22, city='Unspecified'), Person(name='Jack Marston', age=4, city='Unspecified'), Person(name='Bill Williamson', age=33, city='Unspecified'), Person(name='Javier Escuella', age=25, city='Nuevo Paraiso, Mexico'), Person(name='Charles Smith', age=27, city='Canada'), Person(name='Sadie Adler', age=25, city='Unspecified'), Person(name='Micah Bell', age=39, city='Unspecified'), Person(name='Lenny Summers', age=19, city='Unspecified'), Person(name='Sean MacGuire', age=25, city='Ireland'), Person(name='Josiah Trelawny', age=55, city='Unspecified'), Person(name='Leopold Strauss', age=42, city='Vienna, Austria'), Person(name='Karen Jones', age=27, city='Unspecified'), Person(name='Mary-B